In [23]:
from __future__ import annotations

import torch
from torch.autograd.gradcheck import (
    get_analytical_jacobian,
    get_numerical_jacobian,
)

from dxtb import GFN1_XTB as par
from dxtb import Calculator, OutputHandler, labels
from dxtb._src.typing import DD, Callable, Tensor
from dxtb.config import ConfigCache

from samples import samples


DEVICE = torch.device("cpu")

def gradchecker(
    dtype: torch.dtype, name: str, scf_mode: str, scp_mode: str
) -> tuple[Callable[[Tensor], Tensor], Tensor]:
    """Prepare gradient check from `torch.autograd`."""
    dd: DD = {"dtype": dtype, "device": DEVICE}

    sample = samples[name]
    numbers = sample["numbers"].to(DEVICE)
    positions = sample["positions"].to(**dd)

    opts = {
        "scf_mode": scf_mode,
        "scp_mode": scp_mode,
    }

    calc = Calculator(numbers, par, **dd, opts=opts)
    calc.opts.cache = ConfigCache(enabled=False, fock=True)
    OutputHandler.verbosity = 0

    # variables to be differentiated
    pos = positions.clone().requires_grad_(True)

    def func(p: Tensor) -> Tensor:
        _ = calc.get_energy(p)  # triggers Fock matrix computation
        return calc.cache["fock"]

    return func, pos

def analytical_jacobian(fn, inputs):
    # wrap input in a 1‑tuple, flatten output to 1‑D
    inputs_tup = (inputs,)
    y = fn(inputs)
    y_flat = y.reshape(-1)    # shape [M*N]
    # returns a tuple of Jacobians—one per input
    (J_flat,), reentrant, sizes_ok, types_ok = get_analytical_jacobian(
        inputs_tup,
        y_flat,
        nondet_tol=0.0,
        grad_out=1.0,
    )
    # J_flat shape: (inputs.numel(), y_flat.numel())
    return J_flat

def numerical_jacobian(fn, inputs, eps=1e-6):
    # we need a function that takes a tuple of inputs
    # and returns a flat (1‑D) output
    def flat_fn(inp_tuple):
        x = inp_tuple[0]
        y = fn(x)
        return y.reshape(-1)
    # get_numerical_jacobian returns one Jacobian per input
    (J_flat,) = get_numerical_jacobian(
        flat_fn,
        inputs,   # single Tensor; internals call _as_tuple on it
        eps=eps,
    )
    # J_flat shape: (inputs.numel(), y_flat.numel())
    return J_flat


#### Testing ####
SCP_MODE = "potential"

NAME = "MB16_43_01" # Diff of 1e-1 for implicit
NAME = "SiH4" # Diff of 2e-2 
NAME = "LYS_xao" # 

func_impl, pos_impl = gradchecker(torch.float64, NAME, "implicit", SCP_MODE)
func_full, pos_full = gradchecker(torch.float64, NAME, "full", SCP_MODE)

# compute slow/full analytical and numerical
J_an_impl = analytical_jacobian(func_impl, pos_impl); print(J_an_impl.shape)
J_num_impl = numerical_jacobian(func_impl, pos_impl); print(J_num_impl.shape)
J_an_full = analytical_jacobian(func_full, pos_full); print(J_an_full.shape)
J_num_full = numerical_jacobian(func_full, pos_full); print(J_num_full.shape)


/tmp/ipykernel_147046/560074825.py:53: FutureWarning: `get_analytical_jacobian` was part of PyTorch's private API and not meant to be exposed. We are deprecating it and it will be removed in a future version of PyTorch. If you have a specific use for this or feature request for this to be a stable API, please file us an issue at https://github.com/pytorch/pytorch/issues/new
  (J_flat,), reentrant, sizes_ok, types_ok = get_analytical_jacobian(


torch.Size([99, 8836])


/tmp/ipykernel_147046/560074825.py:70: FutureWarning: `get_numerical_jacobian` was part of PyTorch's private API and not meant to be exposed. We are deprecating it and it will be removed in a future version of PyTorch. If you have a specific use for this or feature request for this to be a stable API, please file us an issue at https://github.com/pytorch/pytorch/issues/new
  (J_flat,) = get_numerical_jacobian(


torch.Size([99, 8836])


/tmp/ipykernel_147046/560074825.py:53: FutureWarning: `get_analytical_jacobian` was part of PyTorch's private API and not meant to be exposed. We are deprecating it and it will be removed in a future version of PyTorch. If you have a specific use for this or feature request for this to be a stable API, please file us an issue at https://github.com/pytorch/pytorch/issues/new
  (J_flat,), reentrant, sizes_ok, types_ok = get_analytical_jacobian(


torch.Size([99, 8836])


/tmp/ipykernel_147046/560074825.py:70: FutureWarning: `get_numerical_jacobian` was part of PyTorch's private API and not meant to be exposed. We are deprecating it and it will be removed in a future version of PyTorch. If you have a specific use for this or feature request for this to be a stable API, please file us an issue at https://github.com/pytorch/pytorch/issues/new
  (J_flat,) = get_numerical_jacobian(


torch.Size([99, 8836])


In [24]:
max_diff_impl = torch.max(torch.abs(J_an_impl - J_num_impl))
print(f"Max diff gradients (implicit): {max_diff_impl:.2e}")
max_diff_full = torch.max(torch.abs(J_an_full - J_num_full))
print(f"Max diff gradients (full): {max_diff_full:.2e}")

Max diff gradients (implicit): 1.61e-01
Max diff gradients (full): 2.13e-08


### (Dependent on the sample) the gradients of the implicit version have differences of up to 1e-1!